In [ ]:
import requests
import json

baseURL = "https://api.coingecko.com/api/v3"
landing_folder = "../data/crypto/"

# yg bakal dipake
# endpoint_coins_markets

id_url = "bitcoin"

endpoint_ping = "/ping"
endpoint_simple_price = "/simple/price"
endpoint_supported_vs_currencies = "/simple/supported_vs_currencies"

endpoint_coins_list = "/coins/list"
endpoint_coins_markets = "/coins/markets"
endpoint_coins_id = "/coins/"
endpoint_coins_market_chart = "/coins/" + id_url + "/market_chart"
endpoint_coins_market_chart_range = "/coins/" + id_url + "/market_chart/range"

endpoint_exchanges_list = "/exchanges/list"

params_empty = {}

params_simple_price = {
    "ids": "bitcoin",
    "vs_currencies": "idr",
}

params_coins_markets = {
    "vs_currency": "usd",
    "ids": "bitcoin,ethereum,binancecoin,ripple,solana,cardano,dogecoin,tron",
}

params_coins_id = {
    "id": "bitcoin",
    "tickers": "false",
}

params_coins_market_chart = {
    "vs_currency": "idr",
    "days": "30",
    "interval": "daily",
}

params_coins_market_chart_range = {
    "vs_currency": "idr",
    "from": "1434796113",
    "to": "1687256913",
}

In [ ]:
requestURL = baseURL + endpoint_coins_markets
print(requestURL)

r = requests.get(requestURL, params=params_coins_markets)

print(r.url)

pretty_json = json.loads(r.text)
print(json.dumps(pretty_json, indent=2))

with open(landing_folder + "tes_dump.json", "w") as write_file:
    json.dump(pretty_json, write_file, indent=2) # 3 KB

In [ ]:
import pandas as pd

In [ ]:
df = pd.json_normalize(pretty_json)

In [ ]:
df.dtypes
df.columns

In [ ]:
df[['id', 'symbol', 'name', 'image', 'current_price', 'market_cap',
       'market_cap_rank', 'fully_diluted_valuation', 'total_volume',
       'high_24h', 'low_24h', 'price_change_24h']]
df[['price_change_percentage_24h', 'market_cap_change_24h',
       'market_cap_change_percentage_24h', 'circulating_supply',
       'total_supply', 'max_supply', 'ath', 'ath_change_percentage']]
df[['ath_date', 'atl', 'atl_change_percentage', 'atl_date', 'roi',
       'last_updated', 'roi.times', 'roi.currency', 'roi.percentage']]
# df['image']

In [ ]:
df_exclude = df.drop(columns=['image', 'roi', 'roi.times', 'roi.currency', 'roi.percentage'])

In [ ]:
df_exclude

In [ ]:
df_exclude.dtypes

In [ ]:
df_split = []
coin_list = ['btc', 'eth', 'bnb', 'xrp', 'sol', 'ada', 'doge', 'trx']

# df_exclude.iloc[0:1]

for row in df_exclude.index:
    temp_df = (df_exclude.iloc[row:row+1]).reset_index(drop=True)
    temp_df.to_csv(landing_folder + "tes_dump_" + coin_list[row] + ".csv")
    df_split.append(temp_df)
    # print(df_exclude.iloc[row:row+1])

In [ ]:
type(df_split[1])

In [ ]:
df.to_csv(landing_folder + "tes_dump.csv") # 2 KB
df.to_parquet(landing_folder + "tes_dump_fp.pq", engine="fastparquet") # 9 KB
df.to_parquet(landing_folder + "tes_dump_pa.pq", engine="pyarrow") # 22 KB